# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

### Getting some constants

We get some constants from the local execution environment.

In [5]:
import boto3
import sagemaker
import time
import json
import base64
import requests
import argparse
import os

from boto3.dynamodb.conditions import Key
from pipeline import get_pipeline

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [6]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()
account_number = boto3.client('sts').get_caller_identity().get('Account')
client = boto3.client('sagemaker')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [7]:
model_package_group_name = f"CustomerChurnGenericModel"
pipeline_name = f"CustomerChurnPipeline"
generic_model_data_bucket_name = f"sagemaker-mlaas-pooled-{region}-{account_number}"

## Upload Sample Training Data

In [8]:
s3_bucket_prefix = "sample-data"
local_path = "data/AnyCompany"
sagemaker.Session().upload_data(path=local_path,bucket=generic_model_data_bucket_name,key_prefix=s3_bucket_prefix)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


's3://sagemaker-mlaas-pooled-us-west-2-137751364205/sample-data'

### Get the pipeline instance
Here we get the pipeline instance from your pipeline module so that we can work with it.

In [9]:
pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is interpreted in pipeline execution time only. As the function needs to evaluate the argument value in SDK compile time, the default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.
/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


### Submit the pipeline to SageMaker and start execution
Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [10]:
pipeline.upsert(role_arn=role)

Popping out 'TrainingJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.
Popping out 'ModelPackageName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.


{'PipelineArn': 'arn:aws:sagemaker:us-west-2:137751364205:pipeline/CustomerChurnPipeline',
 'ResponseMetadata': {'RequestId': '2b2e0e21-7ce0-4de0-a2aa-8c94c407c66f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2b2e0e21-7ce0-4de0-a2aa-8c94c407c66f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '89',
   'date': 'Fri, 13 Oct 2023 18:03:09 GMT'},
  'RetryAttempts': 0}}

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [11]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.t3.large",
        ProcessingInstanceCount=1,
        TrainDataPath = f"s3://{generic_model_data_bucket_name}/sample-data/train/train.csv",
        TestDataPath = f"s3://{generic_model_data_bucket_name}/sample-data/test/test.csv",
        ValidationDataPath = f"s3://{generic_model_data_bucket_name}/sample-data/validation/validation.csv",
        ModelPath = f"s3://{generic_model_data_bucket_name}/model_artifacts",
        BucketName = f"{generic_model_data_bucket_name}",
        ObjectKey = f"model_artifacts/basic_tier/output",
        ModelPackageGroupName = f"{model_package_group_name}",
        ModelVersion = "0"
    )
)


### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [12]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-west-2:137751364205:pipeline/CustomerChurnPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-west-2:137751364205:pipeline/CustomerChurnPipeline/execution/mxnav5k9xoj7',
 'PipelineExecutionDisplayName': 'execution-1697220207795',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'customerchurnpipeline',
  'TrialName': 'mxnav5k9xoj7'},
 'CreationTime': datetime.datetime(2023, 10, 13, 18, 3, 27, 735000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 10, 13, 18, 3, 27, 735000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-west-2:137751364205:user-profile/d-z3plqyka7bl0/mlaas-provider-user',
  'UserProfileName': 'mlaas-provider-user',
  'DomainId': 'd-z3plqyka7bl0'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-west-2:137751364205:user-profile/d-z3plqyka7bl0/mlaas-provider-user',
  'UserProfileName': 'mlaas-provider-user',
  'DomainId': 'd-z3plqyka7

We can wait for the execution by invoking `wait()` on the execution. The pipeline execution can take about 10 minutes to be completed. For the purpose of this workshop, a trained model artifact has already been provided in the S3 folder to allow us to proceed with the deployment without waiting for this pipeline to be completed. We will back and check when its executive is completed. 

## Create / Update Basic Tier SageMaker Endpoint

We will deploy the model to a SageMaker dedicated endpoint.
QUESTION: We are preprovisioning the Basic Tier SageMake Endpoint. I guess we need to remove this below block.

In [15]:
timestamp = int(time.time())

model_name=f'GenericModel-{timestamp}'
endpoint_config_name = f"EndpointConfig-{model_name}-{timestamp}"
endpoint_name = "Endpoint-GenericModel"
shared_inference_api_url = "https://lmla2luuak.execute-api.us-east-1.amazonaws.com/" # Example: https://3289dfakjkak.execute-api.us-east-1.amazonaws.com/
model_artifact_file_name = "sample-data.model.0.tar.gz"

# get image URI
image_uri = sagemaker.image_uris.retrieve(
        framework="xgboost",
        region=region,
        version="1.0-1",
        py_version="py3",
        instance_type='ml.t2.medium',
    )

# create sagemaker model
create_model_api_response = client.create_model(
                                    ModelName=model_name,
                                    PrimaryContainer={
                                        'Image': image_uri,
                                        'ModelDataUrl': f"s3://{generic_model_data_bucket_name}/model_artifacts/basic_tier/output/{model_artifact_file_name}",
                                        'Environment': {}
                                    },
                                    ExecutionRoleArn=role
                            )
print ("create_model API response", create_model_api_response)

# create sagemaker endpoint config
update_endpoint_config_api_response = client.create_endpoint_config(
                                            EndpointConfigName=endpoint_config_name,
                                            ProductionVariants=[
                                                {
                                                    'VariantName': 'prod1',
                                                    'ModelName': model_name,
                                                    'InitialInstanceCount': 1,
                                                    'InstanceType': 'ml.t2.medium'
                                                },
                                            ]
                                       )

print ("update_endpoint_config API response", update_endpoint_config_api_response)

# create sagemaker endpoint
update_endpoint_api_response = client.update_endpoint(
                                    EndpointName=endpoint_name,
                                    EndpointConfigName=endpoint_config_name,
                                )

print ("update_endpoint API response", update_endpoint_api_response)    

print(f"Updating endpoint {endpoint_name}...")
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)
print(f"Endpoint {endpoint_name} is in service.")

create_model API response {'ModelArn': 'arn:aws:sagemaker:us-west-2:137751364205:model/genericmodel-1697220827', 'ResponseMetadata': {'RequestId': '651ec2e1-fe33-45d5-b9bf-84eb560eafce', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '651ec2e1-fe33-45d5-b9bf-84eb560eafce', 'content-type': 'application/x-amz-json-1.1', 'content-length': '85', 'date': 'Fri, 13 Oct 2023 18:13:48 GMT'}, 'RetryAttempts': 0}}
update_endpoint_config API response {'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:137751364205:endpoint-config/endpointconfig-genericmodel-1697220827-1697220827', 'ResponseMetadata': {'RequestId': '8836242c-d867-4669-b878-00a235821309', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8836242c-d867-4669-b878-00a235821309', 'content-type': 'application/x-amz-json-1.1', 'content-length': '130', 'date': 'Fri, 13 Oct 2023 18:13:48 GMT'}, 'RetryAttempts': 0}}
update_endpoint API response {'EndpointArn': 'arn:aws:sagemaker:us-west-2:137751364205:endpoint/endpoint-g

## Onboard Basic Tier Tenants

The following Script retrieves the SaaS Control Plane URL and the username and password you require to login.

In [4]:
!chmod +x ../setup/create-admin-user.sh
!../setup/create-admin-user.sh

Admin user created successfully.
Please use admin username: admin-user and password: Mlaa$1234 to login to admin site URL: https://d2yny8jomrd8c0.cloudfront.net


<div class="alert alert-block alert-info">
<font color='black'><b>!!! Follow instructions in the Workshop Studio to onboard 2 (two) Basic Tier Tenants</b></font>
</div>

## Test Inference

Execute the cell below to initialize the inference helper functions

In [16]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
def basic_tier_run_inference(username, password, tenant_name, request):
    api_url=get_api_url(tenant_name)
    jwt = get_jwt(username, password, tenant_name, api_url)
    invoke_inference_endpoint(jwt, request, api_url+'basic_inference')
    
def run_inference(username, password, tenant_name, request):
    api_url=get_api_url(tenant_name)
    jwt = get_jwt(username, password, tenant_name, api_url)
    invoke_inference_endpoint(jwt, request, api_url+'inference')

def get_jwt(username, password, tenant_name, api_url):

    auth_str = f'{username}:{password}'
    byte_str = auth_str.encode('ascii')
    auth_b64 = base64.b64encode(byte_str)

    headers = {
        'Authorization': 'Basic {}'.format(auth_b64.decode('ascii')),
        'tenant-name': tenant_name
    }

    try:
        print("Getting JWT")
        response = requests.get(api_url + 'jwt', headers=headers)
        # print(response.text if response.text else response.reason)
        jwt = json.loads(response.text)['jwt']
        return jwt
    except Exception as e:
        print("Error getting JWT", e)
        
        
    
def invoke_inference_endpoint(jwt, request, api_url):

    headers = {
        'Authorization': 'Bearer {}'.format(jwt),
        'Content-Type': 'text/csv'
    }
    
    try:
        print(f"Inference request with: {request}")
        response = requests.post(api_url, headers=headers, data=request)
        print(response.text if response.text else response.reason)
    except Exception as e:
        print("Error executing inference request", e)

def get_api_url(tenant_name):
    dynamodb = boto3.resource('dynamodb')
    table_tenant_details = dynamodb.Table('MLaaS-TenantDetails')
    
    try:
        
        tenant_details = table_tenant_details.query(
            IndexName='tenantName-index',
            KeyConditionExpression=Key('tenantName').eq(tenant_name)
        )

        api_url = tenant_details['Items'][0]['apiGatewayUrl']
        print(api_url)
        return api_url      
    except Exception as error:
        print(f'[Error]: {error}')
        
    


First let us send an inference request for our first tenant (basic1).

In [24]:
basic_tenant_1_username = "" # Example: mlaas+basic1@amazon.com
basic_tenant_1_name = "" # Example: basic1
basic_tenant_1_request = "84,1,3,98,2,4" 

basic_tier_run_inference(username=basic_tenant_1_username,password="Mlaa$1234",tenant_name=basic_tenant_1_name,request=basic_tenant_1_request)

https://cu03766ja4.execute-api.us-west-2.amazonaws.com/v1/
Getting JWT
{"jwt": "eyJraWQiOiJcL0FoYllcLzgxMEZSdDF2bmo2VzNUMTM4T2xTN2pJYitsWFRcL1k5dVB5K3FRPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiI0ODYxZDNlMC0zMGIxLTcwN2UtZTYzNS1kY2U0OWIwZGJlYjMiLCJjb2duaXRvOmdyb3VwcyI6WyJuYnhjM3ZweHR6ZmZlZWw4em50cXUzIl0sImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy13ZXN0LTIuYW1hem9uYXdzLmNvbVwvdXMtd2VzdC0yX0ZCQkV1MHpWTCIsImNvZ25pdG86dXNlcm5hbWUiOiJtZWhyLm5hamFmaStiYXNpYzFAZ21haWwuY29tIiwib3JpZ2luX2p0aSI6IjdmMzJlYjUyLTQyMTktNDk4MC1hNWY0LWYyZjM5NjgzMTk1OSIsImN1c3RvbTp0ZW5hbnRJZCI6Im5ieGMzdnB4dHpmZmVlbDh6bnRxdTMiLCJhdWQiOiIzMDNlZ3ZuajU0bXRjajMwZ25tbnNqNm10cCIsImV2ZW50X2lkIjoiYTFlZGJkMGEtNTU5Yi00YWVjLTgyZjAtY2ExNDhhZmM3MWViIiwiY3VzdG9tOnVzZXJSb2xlIjoiVGVuYW50QWRtaW4iLCJ0b2tlbl91c2UiOiJpZCIsImF1dGhfdGltZSI6MTY5NzIyNTE3NiwiZXhwIjoxNjk3MjI4Nzc2LCJpYXQiOjE2OTcyMjUxNzYsImp0aSI6Ijk5ZTYxOWRhLTNmNmYtNDkzMy04MDMzLTg1ZGVlYzlhY2M3OSIsImVtYWlsIjoibWVoci5uYWphZmkrYmFzaWMxQGdtYWlsLmNvbSJ9.qHFQ5g3WB2DslOYHZdVU2yKLKC08OLBcC1ADuVb8W7V

Next we send an inference request for our second tenant (basic2).

In [23]:
basic_tenant_2_username = "" # Example: mlaas+basic2@amazon.com
basic_tenant_2_name = "" # Example: basic2
basic_tenant_2_request = "18,0,3,105,2,9"

basic_tier_run_inference(username=basic_tenant_2_username,password="Mlaa$1234",tenant_name=basic_tenant_2_name,request=basic_tenant_2_request)

https://cu03766ja4.execute-api.us-west-2.amazonaws.com/v1/
Getting JWT
{"jwt": "eyJraWQiOiJcL0FoYllcLzgxMEZSdDF2bmo2VzNUMTM4T2xTN2pJYitsWFRcL1k5dVB5K3FRPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiI4OGYxNTMyMC0wMDgxLTcwMTItNzcyZi1mZWY1M2NhNmFlMjYiLCJjb2duaXRvOmdyb3VwcyI6WyI5enhwbnJ3NmNqeGFtcGpzanpzdzMyIl0sImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy13ZXN0LTIuYW1hem9uYXdzLmNvbVwvdXMtd2VzdC0yX0ZCQkV1MHpWTCIsImNvZ25pdG86dXNlcm5hbWUiOiJtZWhyLm5hamFmaStiYXNpYzJAZ21haWwuY29tIiwib3JpZ2luX2p0aSI6ImM1ZTliNDFiLWFiNTQtNGIxYi1hYTRkLTdiYzE0NjE5MjdjYSIsImN1c3RvbTp0ZW5hbnRJZCI6Ijl6eHBucnc2Y2p4YW1wanNqenN3MzIiLCJhdWQiOiIzMDNlZ3ZuajU0bXRjajMwZ25tbnNqNm10cCIsImV2ZW50X2lkIjoiMzBkNzA4NzktZGVjMi00YzQxLWJmYmUtNzcxMmE4NTk4ZWI0IiwiY3VzdG9tOnVzZXJSb2xlIjoiVGVuYW50QWRtaW4iLCJ0b2tlbl91c2UiOiJpZCIsImF1dGhfdGltZSI6MTY5NzIyNTE2MiwiZXhwIjoxNjk3MjI4NzYyLCJpYXQiOjE2OTcyMjUxNjIsImp0aSI6IjYyNmY4YWU5LWFlZDItNDI3Yi1iODRmLTRiOTdiNWQ3Y2Y3MCIsImVtYWlsIjoibWVoci5uYWphZmkrYmFzaWMyQGdtYWlsLmNvbSJ9.V3tuK7xAPBm-Vs-e5oM6Mwy2xCMDyIFWhj8WIVSHmS0

## Lab 2

## Onboard Advanced Tier Tenants

<div class="alert alert-block alert-info">
<font color='black'><b>!!! Follow instructions in the Workshop Studio to onboard 2 (two) Advanced Tier Tenants</b></font>
</div>

## Upload Training Data

Execute the cell below to initialize the file upload helper function

In [80]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0

def upload_file(username, password, tenant_name, file, file_type):
    api_url=get_api_url(tenant_name)
    jwt = get_jwt(username, password, tenant_name, api_url)
    file_name =  os.path.basename(file)
    
    headers = {
        'Authorization': 'Bearer {}'.format(jwt),
        'Content-Type': 'text/csv',
        'file-name': file_name,
        'file-type': file_type
    }

    try:
        print(f"Uploading file: {file}")
        response = requests.put(
            api_url + 'upload', headers=headers, data=open(file, 'rb'))
        print(response.text if response.text else response.reason)
    except Exception as e:
        print("Error uploading file", e)

Run the python function above to upload your advanced tenant training data. This function will handle both, getting a JWT Token and uploading the training data to the correct S3 tenant prefix.

NOTE: Provide the following data before executing the upload_file python function
> First, upload the data for advanced tenant 1

In [81]:
advanced_tenant_1_username = "test+advanced1@amazon.com" 
advanced_tenant_1_name = "advanced1" 
advanced_tenant_1_file = "data/Advanced-Tenant1/advanced1_dataset.csv" 

In [82]:
upload_file(username=advanced_tenant_1_username, password="Mlaa$1234",tenant_name=advanced_tenant_1_name,file_type="csv",file=advanced_tenant_1_file)

https://6u5yivg9zc.execute-api.us-east-1.amazonaws.com/v1/
Getting JWT
{"jwt": "eyJraWQiOiJQaFwvTW5PTGNQMDVPQnNuNVBianFLNk9QMys1UTA1b2l3WHVsZXNuRGZFaz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiIwZDQ1MjA3NS00NmU4LTQ2ODItODY3ZC01MjE3ZTQwZThlMzkiLCJjb2duaXRvOmdyb3VwcyI6WyJka3FxNHRqN2YzcWc5cmh4ZmxoeXd3Il0sImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy1lYXN0LTEuYW1hem9uYXdzLmNvbVwvdXMtZWFzdC0xX0l4RFFtc1EwbyIsImNvZ25pdG86dXNlcm5hbWUiOiJ1andidWtrYSt0ZW5hbnQ0QGFtYXpvbi5jb20iLCJvcmlnaW5fanRpIjoiNWQzYzkyYTktYzA0MS00YWU3LTljNzMtYzgwOThjMzVlYjJjIiwiY3VzdG9tOnRlbmFudElkIjoiZGtxcTR0ajdmM3FnOXJoeGZsaHl3dyIsImF1ZCI6IjJhcG4xdGJqYzhwMW5zdm1tbnY0bnFsamwiLCJldmVudF9pZCI6Ijg3YmRlMjZiLTQyM2ItNDFjNy1iYTE5LWVlOTI4NGU0ZGVlMCIsImN1c3RvbTp1c2VyUm9sZSI6IlRlbmFudEFkbWluIiwidG9rZW5fdXNlIjoiaWQiLCJhdXRoX3RpbWUiOjE2OTY2MTI1MTcsImV4cCI6MTY5NjYxNjExNywiaWF0IjoxNjk2NjEyNTE3LCJqdGkiOiJiMTljNDIyYy03YzUyLTQ5ZDUtODJiOC02MWQ1ZTBiNGE2NTgiLCJlbWFpbCI6InVqd2J1a2thK3RlbmFudDRAYW1hem9uLmNvbSJ9.qsrrZv6-Na6rU5ZTzJK7LrvhSXQmnTH-3iAwtG0whNOIJT4TVZ

NOTE: Provide the following data before executing the upload_file python function

In [116]:
advanced_tenant_1_username = "advanced2@example.com" 
advanced_tenant_1_name = "advanced2" 
advanced_tenant_1_file = "data/Advanced-Tenant2/advanced2_dataset.csv" 

In [107]:
upload_file(username=advanced_tenant_2_username, password="Mlaa$1234",tenant_name=advanced_tenant_2_name,file_type="csv",file=advanced_tenant_2_file)

Getting JWT
Error getting JWT Invalid URL 'v1/jwt': No scheme supplied. Perhaps you meant https://v1/jwt?
Uploading file: tenant-training-data/basic-tenant-1-training-data.csv
Error uploading file [Errno 2] No such file or directory: 'tenant-training-data/basic-tenant-1-training-data.csv'


<div class="alert alert-block alert-info">
<font color='black'><b>!!! Verify that execution of 2 new training pipelines has started. Use SageMaker console to check with execution status. Wait for the execution of the pipelines to complete</b></font>
</div>

## Test Inference

In [83]:
advanced1_request = "18,0,3,105,2,9" 
run_inference(username=advanced_tenant_1_username,password="Mlaa$1234",tenant_name=advanced_tenant_1_name,request=advanced1_request)

https://6u5yivg9zc.execute-api.us-east-1.amazonaws.com/v1/
Getting JWT
{"jwt": "eyJraWQiOiJQaFwvTW5PTGNQMDVPQnNuNVBianFLNk9QMys1UTA1b2l3WHVsZXNuRGZFaz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiIwZDQ1MjA3NS00NmU4LTQ2ODItODY3ZC01MjE3ZTQwZThlMzkiLCJjb2duaXRvOmdyb3VwcyI6WyJka3FxNHRqN2YzcWc5cmh4ZmxoeXd3Il0sImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy1lYXN0LTEuYW1hem9uYXdzLmNvbVwvdXMtZWFzdC0xX0l4RFFtc1EwbyIsImNvZ25pdG86dXNlcm5hbWUiOiJ1andidWtrYSt0ZW5hbnQ0QGFtYXpvbi5jb20iLCJvcmlnaW5fanRpIjoiOWIwNmNhNzctZjE3Ny00MTdmLTg1YmQtMTUwMzU1MmM3OTFhIiwiY3VzdG9tOnRlbmFudElkIjoiZGtxcTR0ajdmM3FnOXJoeGZsaHl3dyIsImF1ZCI6IjJhcG4xdGJqYzhwMW5zdm1tbnY0bnFsamwiLCJldmVudF9pZCI6ImFjNmYzODhkLTM5ZjYtNGU1NS04M2Q2LTAyNDBmOTMyY2NlYSIsImN1c3RvbTp1c2VyUm9sZSI6IlRlbmFudEFkbWluIiwidG9rZW5fdXNlIjoiaWQiLCJhdXRoX3RpbWUiOjE2OTY2MTI1NjQsImV4cCI6MTY5NjYxNjE2NCwiaWF0IjoxNjk2NjEyNTY0LCJqdGkiOiI4MDIzNTAwZi1jOGJiLTRlMTItYWZkNS1jOWQ2NDQzMmM2MjciLCJlbWFpbCI6InVqd2J1a2thK3RlbmFudDRAYW1hem9uLmNvbSJ9.e_mxKuLnZtPpx90HeVlySCXmxjRv8eipAbUPplYwvZrvbHotRE

In [123]:
advanced2_request = "18,0,3,105,2,9" 
run_inference(username=advanced_tenant_2_username,password="Mlaa$1234",tenant_name=advanced_tenant_2_name,request=advanced_tenant_2_request)

Getting JWT
Error getting JWT Invalid URL 'v1/jwt': No scheme supplied. Perhaps you meant https://v1/jwt?
Inference request with: 1993,2244,5,2,1.18,2
Error executing inference request Invalid URL 'v2/inference': No scheme supplied. Perhaps you meant https://v2/inference?


### Lab 3

<div class="alert alert-block alert-info">
<font color='black'><b>!!! Follow instructions in the Workshop Studio to onboard 1 Premium Tier Tenant named Premium1</b></font>
</div>

## Test Inference

In [ ]:
premium_tenant_1_username = "premium1@example.com" 
premium_tenant_1_name = "premium1" 
dedicated_inference_api_url_for_premium1 = ""

In [121]:
premium1_request = "18,0,3,105,2,9" 
run_inference(username=premium_tenant_1_username,password="Mlaa$1234",tenant_name=premium_tenant_1_name,request=request)

Getting JWT
Error getting JWT Invalid URL 'v1/jwt': No scheme supplied. Perhaps you meant https://v1/jwt?
Inference request with: 1993,2244,5,2,1.18,2
Error executing inference request Invalid URL 'v2/inference': No scheme supplied. Perhaps you meant https://v2/inference?


# Lab4

## Test Inference

We have hardcoded the tenant **advanced1** tenant id. Now lets run inference request for different tenant **advanced2**. We will be using **advanced2** tenant users username and tenant name as shown below. If you used different tenant name and username please make sure to use the correct tenant name and username. 

In [ ]:
advanced_tenant_2_username="test+advanced2@example.com"
advanced_tenant_2_name = "advanced2"
advanced_tenant_2_request = "18,0,3,105,2,9" 
run_inference(username=advanced_tenant_2_username,password="Mlaa$1234",tenant_name=advanced_tenant_2_name,request=advanced_tenant_2_request)